In [30]:
import json
from openai import AzureOpenAI
import requests
from dotenv import load_dotenv
import os
from decimal import Decimal

In [2]:
load_dotenv()
print(os.environ.get('AZURE_OPENAI_KEY'))

d213aa73d27b4facb3898449b963ea11


In [3]:
endpoint = os.environ.get('ENDPOINT_URL')
deployment = os.environ.get('DEPLOYMENT_NAME')
api_key = os.environ.get('AZURE_OPENAI_KEY')

In [4]:
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2023-07-01-preview"
)

In [5]:
def call_openai_function_calling(messages, tools):
    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        tools = tools,
        tool_choice="auto",
        
    )
    return response

In [6]:
import mysql.connector

def connect_to_database():
# Establish a connection to the MySQL database
    conn = mysql.connector.connect(
        host="database-1.czjbva8mjsze.ap-south-1.rds.amazonaws.com",
        user="RZCvH6mU954tK",
        password="hDxBO2q08qWrHJRf",
        database="MRP_PROD"
    )
    return conn

In [7]:
def get_table_names(conn):
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES;")
    tables = [table[0] for table in cursor.fetchall()]
    return tables

def get_column_names(conn, table_name):
    cursor = conn.cursor()
    cursor.execute(f"DESCRIBE {table_name};")
    columns = [column[0] for column in cursor.fetchall()]
    return columns

def get_database_info(conn):
    table_dicts = []
    table_names = get_table_names(conn)
    for table_name in table_names:
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

In [8]:
conn = connect_to_database()
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)
print(database_schema_string)


Table: address
Columns: id, city, country, pin_code, state, street, organization
Table: bill
Columns: id, balance_amount, bill_date, bill_number, bill_status, reference_number, source_of_supply, total, organization, purchase_order, supplier, warhouse
Table: bill_row
Columns: id, cost_per_unit, item_total, linked_quantity, quantity, tax_amount, bill, tax, variant
Table: committed
Columns: id, quantity, status, inventory_id, manufacturing_order, manufacturing_order_ingredient, purchase_order, purchase_order_row, sales_order, sales_order_row, committed, committed_serial_number, deadline, expected, expected_serial_number
Table: config
Columns: id, name, material_id, product_id
Table: config_attributes
Columns: id, config_name, config_value, variant_id
Table: config_value
Columns: id, value, config_id
Table: customer
Columns: id, company_name, contact_name, email, gst_number, gst_treatment, is_deleted, opening_balance_amount, outstanding_receivable_amount, phone_number, place_of_contact, un

In [18]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

In [21]:
def ask_database(conn,query):
    conn = connect_to_database()
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    return str(result)


In [11]:
messages = [
    {"role": "system", "content": "You are a helpful assistant connected to a MySQL database."},
    {"role": "user", "content": "Give me the top 3 product pricing"}
]
response = call_openai_function_calling(messages,tools)

response_message = response.choices[0].message

print(response_message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_BGlDsAlpgC0OIRFefh91KCBg', function=Function(arguments='{"query":"SELECT name, unit, sales_price FROM product ORDER BY sales_price DESC LIMIT 3;"}', name='ask_database'), type='function')])


In [34]:
tools_call = response_message.tool_calls

if tools_call:
     tool_call_id = tools_call[0].id
     tool_function_name = tools_call[0].function.name
     tool_query_string = """
     SELECT 
    CASE 
        WHEN p.id IS NOT NULL THEN p.name
        ELSE m.name
    END AS product_name,
    SUM(ir.item_total) AS total_revenue,
    COUNT(DISTINCT i.id) AS number_of_sales
FROM sales_order_row ir
JOIN sales_order i ON ir.sales_order = i.id
LEFT JOIN variant v ON ir.variant = v.id
LEFT JOIN product p ON v.product_id = p.id
LEFT JOIN material m ON v.material_id = m.id
WHERE i.organization = UNHEX(REPLACE("7f366021-c66a-4fdc-99ae-bdfce2113cb2", "-", ""))
    AND i.sales_order_date IS NOT NULL
GROUP BY 
    CASE 
        WHEN p.id IS NOT NULL THEN p.name
        ELSE m.name
    END
ORDER BY total_revenue DESC
LIMIT 10;
     """
     if tool_function_name == 'ask_database':
          results = execute_query(conn,tool_query_string)
          
          messages.append({
            "role":"tool", 
            "tool_call_id":tool_call_id, 
            "name": tool_function_name, 
            "content":json.dumps(results, use_decimal=True)
        })

          model_response = call_openai_function_calling(messages=messages, tools = tools)
          print(model_response.chioces[0].content)
     else: 
            print(f"Error: function {tool_function_name} does not exist")
else:
     print(response_message.content)
         




# By using Chatbot 
# Use Funtion inside the SQL Call the Function 

    
             
         
          
         

TypeError: JSONEncoder.__init__() got an unexpected keyword argument 'use_decimal'

In [23]:
print(ask_database)

<function ask_database at 0x7e191c484670>
